In [26]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener

from bs4 import BeautifulSoup as bs

import time
import pandas as pd

In [27]:
url = 'https://www.ligastavok.ru/bets/live'
driver_path = 'C:\Dev\Drivers\chromedriver.exe'
driver_path = ''

In [58]:
class Bot(object):
    def __init__(self, page='', driver_path='', Listener = None):
        self.page = page
        if Listener:
            self.driver = EventFiringWebDriver(webdriver.Chrome(), MyListener())
        else:
            if driver_path:
                self.driver = webdriver.Chrome(driver_path)
            else:
                self.driver = webdrover.Chrome()
        self.driver.get(self.page)
        self.sport_kind = []
        self.home_team = []
        self.guest_team = []
        self.match_day = []
        self.match_location = []
        self.bet_1 = []
        self.bet_X = []
        self.bet_2 = []

    def parse(self):
        bot = self.driver
        '''путь к контейнеру в котором содержится блоки с результатами спортивных событий'''
        events_block_xpath = '//*[contains(@class, "live__events-block")]'
        
        content = bot.find_element_by_id('content').find_element_by_xpath(events_block_xpath)
#         xpath = '//*[contains(@itemtype, "http://schema.org/Event") and contains(@class, "bui-event-row")]'
        xpath = '//*[@itemtype = "http://schema.org/Event" and contains(@class, "bui-event-row")]'
           
        elems = content.find_elements_by_xpath(xpath)
        count = len(elems)
        i = 1
        for elem in elems:
            print(f'element {i} from {count}: \n\tid = {elem.id}\n\tlocation:{elem.location}')
            
            try: 
                sport_kind = self.get_sport_kind(elem)
                home_team = self.get_home_team(elem)
                guest_team = self.get_guest_team(elem)
                match_day = self.get_match_day(elem)
                match_location = self.get_match_location(elem)
                bet_1 = self.get_bet_1(elem)
                bet_X = self.get_bet_X(elem)
                bet_2 = self.get_bet_2(elem)
                
                self.sport_kind.append(sport_kind)
                self.home_team.append(home_team)
                self.guest_team.append(guest_team)
                self.match_day.append(match_day)
                self.match_location.append(match_location)
                self.bet_1.append(bet_1)
                self.bet_X.append(bet_X)
                self.bet_2.append(bet_2)
                i = i + 1
#                 if i == 5:
#                     break                
            except Exception as e:
                print(f"Ошибка! {e}: {e.args}")
                            

    def do(self, sec=60):
        begintime = time.time()
        while time.time() - begintime < sec:
            print('=' * 50)
            self.parse()
            time.sleep(5)
#         self.driver.quit()
        self.df = pd.DataFrame(
            {                
                'sport_kind': self.sport_kind,
                'home_team': self.home_team,
                'guest_team': self.guest_team,
                'match_day': self.match_day,
                'match_location': self.match_location,
                '1': self.bet_1,
                'X': self.bet_X,
                '2': self.bet_2,
            }
        )
#         print(self.df)
        
    def get_sport_kind(self, element):
        try: 
            xpath = './/*[contains(@class, "game-icon")]'
            elem = element.find_element_by_xpath(xpath)
            return elem.get_attribute('data-icon')
        except Exception as e:
            print(f"Ошибка! get_sport_kind {e}: {e.args}")
            return ''
    
    def get_home_team(self, element):
        try:
            xpath = './/*[@itemprop="name"]'
            elem = element.find_elements_by_xpath(xpath)
            return elem[0].text
        except Exception as e:
            print(f"Ошибка! get_home_team {e}: {e.args}")
            return ''
    
    def get_guest_team(self, element):
        try:
            xpath = './/*[@itemprop="name"]'
            elem = element.find_elements_by_xpath(xpath)
            return elem[1].text
        except Exception as e:
            print(f"Ошибка! get_guest_team {e}: {e.args}")
            return ''
    
    def get_match_day(self, element):
        pass
        return ''
    
    def get_match_location(self, element):
        try:
            xpath = './/*[@itemprop="location"]'
            elem = element.find_element_by_xpath(xpath)
            return elem.text
        except Exception as e:
            print(f"Ошибка! get_match_location {e}: {e.args}")
            return ''
    
    def get_bet_1(self, element):
        try:
            xpath = './/*[@itemprop="offers"]'
            elem = element.find_elements_by_xpath(xpath)
            return elem[0].text if len(elem) > 0 else ''
        except Exception as e:
            print(f"Ошибка! get_bet_1 {e}: {e.args}")
            return ''
    
    def get_bet_2(self, element):
        try:
            xpath = './/*[@itemprop="offers"]'
            elem = element.find_elements_by_xpath(xpath)
            if len(elem) == 0:
                return ''
            return elem[2].text if len(elem) >= 3 else elem[1].text
        except Exception as e:
            print(f"Ошибка! get_bet_2 {e}: {e.args}")
            return ''
    
    def get_bet_X(self, element):
        try:
            xpath = './/*[@itemprop="offers"]'
            elem = element.find_elements_by_xpath(xpath)
            return elem[1].text if len(elem) >= 3 else ''
        except Exception as e:
            print(f"Ошибка! get_bet_X {e}: {e.args}")
            return ''

In [59]:
if __name__ == '__main__':
    # driver = webdriver.Chrome()
    # edriver = EventFiringWebDriver(driver, MyListener())
    # edriver.get(url)
    # content = edriver.find_element_by_id('content')
    # xpath = '//*[contains(@itemtype, "http://schema.org/Event") and contains(@class, "bui-event-row")]'
    # elems = driver.find_elements_by_xpath(xpath)
    # for elem in elems:
    #     print(elem.text)
    # # while edriver:
    # #     pass
    # edriver.quit()
    bot = Bot(page=url, driver_path=driver_path)
    bot.do(sec=5)

element 1 from 42: 
	id = 191b2c4b-26e2-4fbb-8ce7-0415e9d010f3
	location:{'x': 264, 'y': 200}
element 2 from 42: 
	id = 68620d22-1ac1-4063-b1f8-534a895544d8
	location:{'x': 264, 'y': 281}
element 3 from 42: 
	id = 969c8c4c-31e9-49ae-ac70-19376fc5c683
	location:{'x': 264, 'y': 362}
element 4 from 42: 
	id = 66459806-e3e6-4ca0-8b30-f26aee342c9f
	location:{'x': 264, 'y': 443}
element 5 from 42: 
	id = 50ee843a-0547-4bf5-9b64-1af3fdb718dc
	location:{'x': 264, 'y': 524}
element 6 from 42: 
	id = 2fe35c19-54fc-458e-a467-102ee1b5043a
	location:{'x': 264, 'y': 605}
element 7 from 42: 
	id = ce00f921-6136-4c6f-80f4-98d47e1f3510
	location:{'x': 264, 'y': 686}
element 8 from 42: 
	id = 6453729b-3d9c-4bf0-b9aa-9f3d85da3535
	location:{'x': 264, 'y': 767}
element 9 from 42: 
	id = be576fca-dcaa-4318-9de0-aa359dbfd0da
	location:{'x': 264, 'y': 848}
element 10 from 42: 
	id = 9e3b02aa-29e3-4f33-a5b3-8461f47620b7
	location:{'x': 264, 'y': 929}
element 11 from 42: 
	id = d353a7d2-ee3f-431a-995f-d70ac2bd

In [35]:
df = pd.DataFrame(
            {                
                'sport_kind': bot.sport_kind[:35],
                'home_team': bot.home_team[:35],
                'guest_team': bot.guest_team[:35],
                'match_day': bot.match_day[:35],
                'location': bot.location[:35],
                '1': bot.bet_1[:35],
                'X': bot.bet_X[:35],
                '2': bot.bet_2[:35],
            }
        )

In [60]:
bot.df

,sport_kind,home_team,guest_team,match_day,match_location,1,X,2
0,football,УГЛОВЫЕ Иордания (мол),УГЛОВЫЕ Вьетнам (мол),,Международные. Молодёжные. Чемпионат Азии. До ...,"2,07","6,24","2,19"
1,football,Акаба,Аль Хуссейн,,Международные. Клубы. Товарищеские матчи (форм...,"2,50","3,00","2,75"
2,football,Теренгану,Свайриенг,,Международные. Клубы. Товарищеские матчи (форм...,,,
3,football,Виктория Кельн,Астра Джурджу,,Международные. Клубы. Товарищеские матчи (форм...,"2,00","4,00","2,88"
4,football,Шабаб Аль Ордон,Аль Рамта,,Международные. Клубы. Товарищеские матчи (форм...,,,
5,football,Кардифф Сити (мол),Колчестер (мол),,Англия. PDL. До 23 лет,"2,18","3,76","2,67"
6,football,Кру (мол),Лидс (мол),,Англия. PDL. До 23 лет,"3,67","4,13","1,71"
7,football,Чарльтон (мол),Миллуолл (мол),,Англия. PDL. До 23 лет,"2,38","3,84","2,38"
8,football,Ирони Тверия,Маккаби Ирони Кирьят Ата,,Израиль. Лига Алеф,"1,20","5,00","19,00"
9,football,Маккаби Цур Шалом,Хапоэль Бней Залафа,,Израиль. Лига Алеф,"7,00","3,75","1,44"
